In [1]:
! mkdir -p 'data/srishty/'
! curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -o 'data/srishty/srishty_essay.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75042  100 75042    0     0   134k      0 --:--:-- --:--:-- --:--:--  135k


In [1]:
from dotenv import load_dotenv
load_dotenv('/Users/srishtysuman/.env')
print(load_dotenv('/Users/srishtysuman/.env'))

True


In [2]:
from llama_index.node_parser import SimpleNodeParser
from llama_index import SimpleDirectoryReader
from llama_index import VectorStoreIndex, ServiceContext, StorageContext
from llama_index.vector_stores import DeepLakeVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms import OpenAI

Create the Llama-index nodes/chunks

In [3]:
#  SimpleDirectoryReader: Load files from file directory and Automatically select the best file reader given file extensions.
documents = SimpleDirectoryReader("./data/srishty/").load_data()
# documents

In [4]:
# SimpleNodeParser deprecated by SentenceSplitter: Parse text with a preference for complete sentences.
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
# node_parser

In [5]:
# get_nodes_from_documents: Parse documents into nodes.
nodes = node_parser.get_nodes_from_documents(documents)
# nodes

In [22]:
type(nodes[0])

llama_index.schema.TextNode

In [6]:
# By default, the node/chunks ids are set to random uuids. To ensure same id's per run, we manually set them.
for idx, node in enumerate(nodes):
    node.id_ = f"node_{idx}"

In [7]:
print(f"Number of Documents: {len(documents)}")
print(f"Number of nodes: {len(nodes)} with the current chunk size of {node_parser.chunk_size}")

Number of Documents: 1
Number of nodes: 57 with the current chunk size of 512


Create a local Deep Lake vector store 

In [8]:
# Create a DeepLakeVectorStore locally to store the vectors
dataset_path = "./data/srishty/deep_lake_db"
vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=True)


In [9]:
# LLM that will answer questions with the retrieved context
llm = OpenAI(model="gpt-3.5-turbo-1106")
embed_model = OpenAIEmbedding()


In [ ]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm,)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [10]:
vector_index = VectorStoreIndex(nodes, service_context=service_context, storage_context=storage_context, show_progress=True)

/Users/srishtysuman/anaconda3/envs/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings:   0%|          | 0/57 [00:00<?, ?it/s]

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-6bM4w***************************************B3IC. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

Upload the local Vectore Store to Activeloop's platform and convert it into a managed database.

In [28]:
local = "./data/srishty/deep_lake_db"
hub_path = "hub://srishtysuman2919/optimization_srishty"
hub_managed_path = "hub://genai360/optimization_srishty_managed"

In [48]:
# First upload our local vector store
deeplake.deepcopy(local, hub_path, overwrite=True, token=ACTIVELOOP_TOKEN, public=True)

Copying dataset: 38%|███▊      | 5/13 [10:55<17:28
Generating embeddings:  98%|█████████▊| 56/57 [12:42<00:13, 13.61s/it]
Copying dataset: 100%|██████████| 13/13 [00:16<00:00


This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/srishtysuman2919/optimization_srishty
Your Deep Lake dataset has been successfully created!


Dataset(path='hub://srishtysuman2919/optimization_srishty', tensors=['embedding', 'id', 'metadata', 'text'])

In [50]:
ds = deeplake.load('hub://srishtysuman2919/optimization_srishty')

-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/srishtysuman2919/optimization_srishty



|

hub://srishtysuman2919/optimization_srishty loaded successfully.



In [49]:
# Create a managed vector store under a different name
deeplake.deepcopy(hub_path, hub_managed_path, overwrite=True, runtime={"tensor_db": True}, token=ACTIVELOOP_TOKEN, public=True)


TokenPermissionError: A dataset does not exist at the specified path, or you do not have sufficient permissions to load or create one. Please check the dataset path and make sure that you have sufficient permissions to the path.

Instantiate a Vector Store with the managed dataset that we just created.

In [45]:
db = DeepLakeVectorStore(dataset_path=hub_managed_path, overwrite=False, read_only=True, token=ACTIVELOOP_TOKEN)

Deep Lake Dataset in hub://srishtysuman2919/optimization_srishty already exists, loading from the storage


Fetching our docs and ids from the vector store.

In [46]:
# Fetch dataset docs and ids 
docs = db.vectorstore.dataset.text.data(fetch_chunks=True, aslist=True)['value']
ids = db.vectorstore.dataset.id.data(fetch_chunks=True, aslist=True)['value']
print(len(docs))

IndexError: index 0 is out of bounds for axis 0 with size 0